In [445]:
import pandas as pd
from datetime import datetime
import pyeto
import numpy as np
import ast       
import math
import xlrd
from ast import literal_eval  
from pandas import DataFrame
from scipy.interpolate import interp1d
import dateutil     #dateutil module provides powerful extensions to the standard datetime module
from dateutil import parser  #This module offers reads the given date in string and convert it to date format or timestamps,it represent a date and/or time from most known formats 
import os
from tkinter import filedialog, messagebox
import matplotlib.pyplot as plt
import folium
import branca.colormap as cm
import json
from IPython.display import display, Markdown, HTML
#from plotly.offline import iplot, init_notebook_mode
#init_notebook_mode()

#import plotly.graph_objs as go

# note that pyeto is available here https://github.com/woodcrafty/PyETo.git
from pyeto import fao

%matplotlib inline

math.exp = np.exp
math.pow = np.power
math.sqrt = np.sqrt

In [446]:
#select scenario for analysis
#0 for average scenario
#1 for worst case scenario
#2 future scenario
s = 1

In [447]:
# Crop name and scenario
# cropname = "Maize"
# scenarioname = "2017_10km"



In [448]:
# Import data 

if s == 1:
    df = pd.read_csv('worst_scenario.csv')
elif s == 2:
    df = pd.read_csv('future_scenario.csv')
else:
    df = pd.read_csv('average_scenario.csv')

df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,pvout_6,pvout_7,pvout_8,pvout_9,pvout_10,pvout_11,pvout_12,c_shallow,c_deep,protected
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,97.750424,94.801152,111.015729,126.503899,118.193017,117.762203,145.751229,0.845,0.76,1
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,97.651525,94.804831,111.183339,126.818136,117.651042,118.024068,145.782492,0.845,0.76,1
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,97.312433,94.384946,111.213198,127.072433,117.729901,119.339730,146.743946,0.845,0.76,1
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,96.911949,93.786559,110.829992,127.168474,117.356280,120.009152,148.191296,0.845,0.76,1
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,100.381859,98.043398,113.597169,128.413275,121.520823,120.468584,146.196274,0.845,0.76,1


In [449]:
#Defining function for estimating parameters required to calculate monthly reference evapotranspiration
def evap_i(lat,elev,wind,srad,tmin,tmax,tavg,month):
    if month ==1:
        J = 15
    else:
        J = 15 + (month-1)*30
        
    latitude = pyeto.deg2rad(lat) #converting latitude values from degrees to radians
    atmosphericVapourPressure = pyeto.avp_from_tmin(tmin) #obtaining atmospheric vapour pressure from t_min
    saturationVapourPressure = pyeto.svp_from_t(tavg) #obtaining saturation vapour pressure from t_avg
    ird = pyeto.inv_rel_dist_earth_sun(J) #obtaining irradiance based on distance from sun?
    solarDeclination = pyeto.sol_dec(J) #solar declination for various months?
    sha = [pyeto.sunset_hour_angle(l, solarDeclination) for l in latitude]
    extraterrestrialRad = [pyeto.et_rad(x, solarDeclination,y,ird) for x, y in zip(latitude,sha)] #extraterrestial radiation
    clearSkyRad = pyeto.cs_rad(elev,extraterrestrialRad) #clear sky radiation
    netInSolRadnet = pyeto.net_in_sol_rad(srad*0.001, albedo=0.23) #net shortwave irradiation
    netOutSolRadnet = pyeto.net_out_lw_rad(tmin, tmax, srad*0.001, clearSkyRad, atmosphericVapourPressure) #net longwave radiation
    netRadiation = pyeto.net_rad(netInSolRadnet,netOutSolRadnet) #net radiation
    tempKelvin = pyeto.celsius2kelvin(tavg) #converting average temperature to Kelvin
    windSpeed2m = wind #wind speed
    slopeSvp = pyeto.delta_svp(tavg) #calculating slope vapour pressure
    atmPressure = pyeto.atm_pressure(elev) #calculating atmospheric pressure from elevation value
    psyConstant = pyeto.psy_const(atmPressure) #calculating psychrometric constant from atmospheric pressure
    
    #the function to use FAO Penman-Monteith equation and return the value of reference evapotranspiration
    return pyeto.fao56_penman_monteith(netRadiation, tempKelvin, windSpeed2m, saturationVapourPressure, 
                                       atmosphericVapourPressure, slopeSvp, psyConstant, shf=0.0)


#initiating the value of reference evapotranspiration
for i in range(1,13):
    df['ETo_{}'.format(i)]=0  #To make sure that it is reset to zero

#calling the function to calculate ETo for each row for each month 
for i in range(1,13):
    df['ETo_{}'.format(i)] = evap_i(df['lat'],df['elevation'],df['wind_{}'.format(i)],df['srad_{}'.format(i)], 
                                    df['tmin_{}'.format(i)],df['tmax_{}'.format(i)],df['tavg_{}'.format(i)],i)


In [450]:
df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,ETo_3,ETo_4,ETo_5,ETo_6,ETo_7,ETo_8,ETo_9,ETo_10,ETo_11,ETo_12
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,6.680300,5.403573,4.966869,4.503674,4.145088,4.842748,5.608238,5.024672,5.034487,5.432109
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,6.741490,5.422991,5.006311,4.525563,4.208431,4.856318,5.668986,5.026697,5.053231,5.449487
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,6.792080,5.419250,5.011598,4.525846,4.221017,4.851568,5.697249,5.049729,5.085405,5.474107
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,6.782639,5.420654,5.004725,4.552273,4.254496,4.852122,5.720390,5.040919,5.074612,5.439782
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,6.828204,5.543967,5.106274,4.683964,4.316969,4.960178,5.723494,5.243543,5.175164,5.584905


In [451]:
# Estimate monthly crop evaropotransoration ETc
for i in range(1,13):
    df['ETc_{}'.format(i)] = df['ETo_{}'.format(i)] * df['kc_{}'.format(i)]

df

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,ETc_3,ETc_4,ETc_5,ETc_6,ETc_7,ETc_8,ETc_9,ETc_10,ETc_11,ETc_12
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,2.672120,5.943930,5.960243,5.404409,3.730579,0.0,2.243295,5.527140,6.041384,6.518531
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,2.696596,5.965290,6.007573,5.430676,3.787588,0.0,2.267595,5.529367,6.063877,6.539385
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,2.716832,5.961175,6.013917,5.431016,3.798915,0.0,2.278900,5.554702,6.102487,6.568929
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,2.713056,5.962719,6.005670,5.462728,3.829046,0.0,2.288156,5.545011,6.089535,6.527739
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,2.731282,6.098363,6.127529,5.620757,3.885272,0.0,2.289398,5.767897,6.210196,6.701885
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6049,Kenya,Laikipia,100,6134,36.824311,0.415420,10000,1711.681942,2.939079,25500.26637,...,2.498273,5.972495,6.407944,5.972014,4.148409,0.0,2.411541,6.168978,6.237966,6.203505
6050,Kenya,Kajiado,23,6135,36.841061,-2.563604,2300,1280.228731,2.078238,22775.98586,...,2.374710,5.525520,5.183370,4.759823,3.689950,0.0,2.145414,6.180726,6.193718,6.127557
6051,Kenya,Kajiado,100,6136,36.824311,-2.496446,10000,1470.620210,1.952978,22033.22939,...,2.256176,5.244972,5.024426,4.631953,3.561526,0.0,2.039698,5.836972,5.764127,5.580387
6052,Kenya,Kajiado,100,6137,36.824311,-2.399384,10000,1370.762115,1.774332,22489.47934,...,2.351821,5.518773,5.266540,4.830254,3.725652,0.0,2.114188,6.105462,6.078514,5.981322


In [452]:
# Calculating the annual precipitation: which is the sum of precipitation values
df['precipitation_annual']=df.filter(like='prec_').sum(axis=1)  #Filter is used to specify the column of interest

df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,ETc_4,ETc_5,ETc_6,ETc_7,ETc_8,ETc_9,ETc_10,ETc_11,ETc_12,precipitation_annual
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,5.943930,5.960243,5.404409,3.730579,0.0,2.243295,5.527140,6.041384,6.518531,183.391199
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,5.965290,6.007573,5.430676,3.787588,0.0,2.267595,5.529367,6.063877,6.539385,172.626264
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,5.961175,6.013917,5.431016,3.798915,0.0,2.278900,5.554702,6.102487,6.568929,162.472554
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,5.962719,6.005670,5.462728,3.829046,0.0,2.288156,5.545011,6.089535,6.527739,154.220479
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,6.098363,6.127529,5.620757,3.885272,0.0,2.289398,5.767897,6.210196,6.701885,206.128980


In [453]:
#calculating the effective rainfall
#the rainfall received is multiplied with relevant factor
#for this case maize is considered a shallow rooted crop

#Initiate
for i in range(1,13):
    df['eff_{}'.format(i)]=0


#the effective rainfall kept with bounds of 0.0001 to ETc value to avoid negative numbers when the two are
#subtracted
for i in range(1,13):
    df['eff_{}'.format(i)] = df['c_shallow'] * df['prec_{}'.format(i)]
    df.loc[df['eff_{}'.format(i)] < 0, 'eff_{}'.format(i)] = 0.0001
    df.loc[(df['eff_{}'.format(i)] >= df['prec_{}'.format(i)]), 'eff_{}'.format(i)] = df['prec_{}'.format(i)]
    df.loc[(df['eff_{}'.format(i)] >= df['ETc_{}'.format(i)]), 'eff_{}'.format(i)] = df['ETc_{}'.format(i)]

In [454]:
#Net Irrigation requirements (IRn) (mm/month)
#(monthly ETc - monthly effective rainfall)*30 days
for i in range (1,13):
    df['IRn_{}'.format(i)]= df['ETc_{}'.format(i)]*30 - df['eff_{}'.format(i)]*30

df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,IRn_3,IRn_4,IRn_5,IRn_6,IRn_7,IRn_8,IRn_9,IRn_10,IRn_11,IRn_12
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,0.0,0.0,0.0,115.079077,0.000000,0.0,1.769271,0.0,0.0,0.000000
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,0.0,0.0,0.0,123.027098,7.910160,0.0,10.653101,0.0,0.0,0.000000
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,0.0,0.0,0.0,133.230382,16.198781,0.0,16.067971,0.0,0.0,21.732228
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,0.0,0.0,0.0,135.406959,15.341630,0.0,19.647147,0.0,0.0,36.998402
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,0.0,0.0,0.0,84.696586,0.000000,0.0,0.000000,0.0,0.0,0.000000


In [455]:
#Peak Crop Water Requirements (PCWR)

# Converting IRn into (m3/ha per month) 
for i in range (1,13):
    df['IRn_{}'.format(i)] *= 10    #0.001*10000 --- 1 hectare*1mm
    df['Monthly_IRn_{}'.format(i)] = df['IRn_{}'.format(i)]
    
# Converting IRn into (m3/ha per day)
for i in range (1,13):
    df['IRnd_{}'.format(i)] = df['IRn_{}'.format(i)] / 30
    
# Peak crop water requirement (PCWR) taken to be equal to net irrigation requirement
for i in range (1,13):
    df['PCWR_{}'.format(i)] = df['IRnd_{}'.format(i)] 
    
# Converting PCWR into  l/s/ha "Duty" - litre/second/hectare
for i in range (1,13):
    df['PCWR_{}'.format(i)] *= 0.012 #1000/86400 (seconds in a day)
    

df.head(5)


,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,PCWR_3,PCWR_4,PCWR_5,PCWR_6,PCWR_7,PCWR_8,PCWR_9,PCWR_10,PCWR_11,PCWR_12
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,0.0,0.0,0.0,0.460316,0.000000,0.0,0.007077,0.0,0.0,0.000000
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,0.0,0.0,0.0,0.492108,0.031641,0.0,0.042612,0.0,0.0,0.000000
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,0.0,0.0,0.0,0.532922,0.064795,0.0,0.064272,0.0,0.0,0.086929
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,0.0,0.0,0.0,0.541628,0.061367,0.0,0.078589,0.0,0.0,0.147994
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,0.0,0.0,0.0,0.338786,0.000000,0.0,0.000000,0.0,0.0,0.000000


In [456]:
#max pcwr
df['pcwr_max']=df.filter(like='PCWR_').max(axis=1) 
df['pwcr_per_day'] = df['pcwr_max']*60*60*24 #pwcr per day per hectare
df

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,PCWR_5,PCWR_6,PCWR_7,PCWR_8,PCWR_9,PCWR_10,PCWR_11,PCWR_12,pcwr_max,pwcr_per_day
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,0.0,0.460316,0.000000,0.0,0.007077,0.0,0.0,0.000000,0.489077,42256.227871
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,0.0,0.492108,0.031641,0.0,0.042612,0.0,0.0,0.000000,0.514527,44455.112280
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,0.0,0.532922,0.064795,0.0,0.064272,0.0,0.0,0.086929,0.561741,48534.438837
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,0.0,0.541628,0.061367,0.0,0.078589,0.0,0.0,0.147994,0.586604,50682.558592
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,0.0,0.338786,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.401216,34665.039135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6049,Kenya,Laikipia,100,6134,36.824311,0.415420,10000,1711.681942,2.939079,25500.26637,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000
6050,Kenya,Kajiado,23,6135,36.841061,-2.563604,2300,1280.228731,2.078238,22775.98586,...,0.0,0.446642,0.423140,0.0,0.145080,0.0,0.0,0.000000,0.446642,38589.885345
6051,Kenya,Kajiado,100,6136,36.824311,-2.496446,10000,1470.620210,1.952978,22033.22939,...,0.0,0.407205,0.401122,0.0,0.120154,0.0,0.0,0.000000,0.407205,35182.505622
6052,Kenya,Kajiado,100,6137,36.824311,-2.399384,10000,1370.762115,1.774332,22489.47934,...,0.0,0.443883,0.423526,0.0,0.128562,0.0,0.0,0.000000,0.443883,38351.470441


In [457]:
#Peak Water Demand (PWD) in l/s 

# PWD = PCWR / Irrigation efficiency(IrrEff) 
# IrrEff = Field Application Efficiency (aeff) * Distribution Efficiency (deff)*100 
# deff = (Conveyance efficiency + field canal efficiency)
# deff: 0.95 (all scenarios)
# aeff: 0.6 (Surface Irr), 0.75 (Sprinkler Irr), 0.9 (Drip Irr)

#efficiency of drip irrigation will be used 

pumping_hours_per_day=8    # Assumption  
deff= 0.95                 # Assumption
aeff= 0.9                 # Assumption

#peak water demand per day in l/s/ha
for i in range (1,13):
    df["PWD_{}".format(i)]= df["PCWR_{}".format(i)]*24/(pumping_hours_per_day*aeff*deff)

df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,PWD_3,PWD_4,PWD_5,PWD_6,PWD_7,PWD_8,PWD_9,PWD_10,PWD_11,PWD_12
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,0.0,0.0,0.0,1.615145,0.000000,0.0,0.024832,0.0,0.0,0.000000
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,0.0,0.0,0.0,1.726696,0.111020,0.0,0.149517,0.0,0.0,0.000000
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,0.0,0.0,0.0,1.869900,0.227351,0.0,0.225515,0.0,0.0,0.305014
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,0.0,0.0,0.0,1.900449,0.215321,0.0,0.275749,0.0,0.0,0.519276
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,0.0,0.0,0.0,1.188724,0.000000,0.0,0.000000,0.0,0.0,0.000000


In [458]:
#pump diameter depending on peak water demand
#diameter in metres
df['PWD_max']=df.filter(like='PWD_').max(axis=1)

df['diam'] = np.sqrt((4*df['PWD_max']/1000)/(3.142*1.5))



df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,PWD_5,PWD_6,PWD_7,PWD_8,PWD_9,PWD_10,PWD_11,PWD_12,PWD_max,diam
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,0.0,1.615145,0.000000,0.0,0.024832,0.0,0.0,0.000000,1.716059,0.038163
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,0.0,1.726696,0.111020,0.0,0.149517,0.0,0.0,0.000000,1.805357,0.039144
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,0.0,1.869900,0.227351,0.0,0.225515,0.0,0.0,0.305014,1.971022,0.040900
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,0.0,1.900449,0.215321,0.0,0.275749,0.0,0.0,0.519276,2.058259,0.041796
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,0.0,1.188724,0.000000,0.0,0.000000,0.0,0.0,0.000000,1.407774,0.034566


In [459]:
#Estimating Energy and Power Requirements

In [460]:
#Total dynamic head (TDH) for ground and surface water sources

kf = 0.013 #friction factor

# #friction head of groundwater
frict_head_gw = (4*kf*8*df["gw_depth"]*(df["PWD_max"]/1000)**2)/(((df["diam"])**5)*(3.142**2)*9.81)
pres_head_gw = 2 #pressure head



pres_head_sw = 2
frict_head_sw = (4*kf*8*df['sw_dist']*(df['PWD_max']/1000)**2)/(((df['diam'])**5)*(3.142**2)*9.81)

#groundwater dynamic head
df['tdh_gw']=df['gw_depth'] + pres_head_gw + frict_head_gw



#checking if water us flowing downhill or uphill
df['tsh'] = df['sw_depth'] - df['elevation']

#surface water dynamic head
df.loc[df['tsh'] <= 0, 'tdh_sw'] = (abs(df['sw_depth'] - df['elevation'])) + pres_head_sw + frict_head_sw
df.loc[df['tsh'] > 0, 'tdh_sw'] = pres_head_sw + frict_head_sw


df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,PWD_8,PWD_9,PWD_10,PWD_11,PWD_12,PWD_max,diam,tdh_gw,tsh,tdh_sw
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,0.0,0.024832,0.0,0.0,0.000000,1.716059,0.038163,117.625701,8.200878,32650.340532
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,0.0,0.149517,0.0,0.0,0.000000,1.805357,0.039144,117.234353,-11.874556,30224.968612
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,0.0,0.225515,0.0,0.0,0.305014,1.971022,0.040900,116.580079,-32.220345,27399.424864
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,0.0,0.275749,0.0,0.0,0.519276,2.058259,0.041796,116.267754,-65.740840,25333.507996
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,0.0,0.000000,0.0,0.0,0.000000,1.407774,0.034566,119.251984,78.915220,43397.812079


In [461]:
#Estimating power (kW)

In [462]:
#Setting the default value for these parameters
for i in range (1,13):
    df['PD_E_gw_{}'.format(i)]=0      #PD_E_gw: Peak Demand (kw) using electric powered pump for ground water
    df['PD_E_sw_{}'.format(i)]=0      #PD_E_sw: Peak Demand (kw) using electric powered pump for surface water 

    
pump_plant_eff=0.57

#calculating the monthly peak (kW) demand
for i in range (1,13):
    PWD = 'PWD_{}'.format(i)
    PD_E_gw = 'PD_E_gw_{}'.format(i)
    PD_E_sw = 'PD_E_sw_{}'.format(i)

    df[PD_E_gw]=(9.81*(df[PWD]/1000)*df['tdh_gw'])/pump_plant_eff #PWD here is in l/s and not m3/s
    df[PD_E_sw]=(9.81*(df[PWD]/1000)*df['tdh_sw'])/pump_plant_eff

df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,PD_E_gw_8,PD_E_sw_8,PD_E_gw_9,PD_E_sw_9,PD_E_gw_10,PD_E_sw_10,PD_E_gw_11,PD_E_sw_11,PD_E_gw_12,PD_E_sw_12
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,0.0,0.0,0.050270,13.953766,0.0,0.0,0.0,0.0,0.000000,0.000000
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,0.0,0.0,0.301676,77.776999,0.0,0.0,0.0,0.0,0.000000,0.000000
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,0.0,0.0,0.452475,106.343699,0.0,0.0,0.0,0.0,0.611981,143.831823
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,0.0,0.0,0.551783,120.227583,0.0,0.0,0.0,0.0,1.039086,226.405820
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000


In [463]:
#Decision between ground or surface water irrigation

In [464]:
df['Max_PD_E_gw']=df.filter(like='PD_E_gw_').max(axis=1) #highest power for gw
df['Max_PD_E_sw']=df.filter(like='PD_E_sw_').max(axis=1)  #highest power for sw
df['PD_E'] = np.minimum.reduce(df[['Max_PD_E_gw', 'Max_PD_E_sw']].values, axis=1) #choosing lowest of the two

#noting whether surface water or groundwater was used
#1 if surface water is chosen and 0 if groundwater is chosen
df.loc[df['Max_PD_E_sw'] < df['Max_PD_E_gw'], 'sw'] = 1
df.loc[df['Max_PD_E_sw'] > df['Max_PD_E_gw'], 'sw'] = 0

df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,PD_E_gw_10,PD_E_sw_10,PD_E_gw_11,PD_E_sw_11,PD_E_gw_12,PD_E_sw_12,Max_PD_E_gw,Max_PD_E_sw,PD_E,sw
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,0.0,0.0,0.0,0.0,0.000000,0.000000,3.473989,964.304049,3.473989,0.0
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,0.0,0.0,0.0,0.0,0.000000,0.000000,3.642606,939.124389,3.642606,0.0
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,0.0,0.0,0.0,0.0,0.611981,143.831823,3.954667,929.452090,3.954667,0.0
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,0.0,0.0,0.0,0.0,1.039086,226.405820,4.118636,897.406916,4.118636,0.0
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,0.0,0.0,0.0,0.0,0.000000,0.000000,2.889301,1051.465625,2.889301,0.0


In [465]:
#Estimating the yearly electricity demand

#total sum of monthly
#months assumed to have same length so this can be added directly
df['PD_E_gw_sum']=df.filter(like='PD_E_gw_').sum(axis=1) #sum power for gw
df['PD_E_sw_sum']=df.filter(like='PD_E_sw_').sum(axis=1)  #sum power for sw

df['PD_sum'] = np.minimum.reduce(df[['PD_E_gw_sum', 'PD_E_sw_sum']].values, axis=1) #choosing the lowest power sum

#monthy energy demand
for i in range (1,13):
    month_demand = 'month_demand_{}'.format(i)
    power_sw = 'PD_E_sw_{}'.format(i)
    power_gw = 'PD_E_gw_{}'.format(i)
    df.loc[df['Max_PD_E_sw'] < df['Max_PD_E_gw'], month_demand] = df[power_sw]*8*30
    df.loc[df['Max_PD_E_sw'] > df['Max_PD_E_gw'], month_demand] = df[power_gw]*8*30
    

#total annual energy consumed in irrigation
df['Annual_elec_demand'] = df['PD_sum'] * 8 * 30

df

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,month_demand_4,month_demand_5,month_demand_6,month_demand_7,month_demand_8,month_demand_9,month_demand_10,month_demand_11,month_demand_12,Annual_elec_demand
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,0.0,0.0,784.727948,0.000000,0.0,12.064718,0.0,0.0,0.000000,1630.550146
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,0.0,0.0,836.134605,53.760176,0.0,72.402149,0.0,0.0,0.000000,1836.522279
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,0.0,0.0,900.426239,109.478088,0.0,108.594018,0.0,0.0,146.875422,2214.493818
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,0.0,0.0,912.684748,103.407326,0.0,132.427845,0.0,0.0,249.380666,2386.373117
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,0.0,0.0,585.533899,0.000000,0.0,0.000000,0.0,0.0,0.000000,1278.966248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6049,Kenya,Laikipia,100,6134,36.824311,0.415420,10000,1711.681942,2.939079,25500.26637,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
6050,Kenya,Kajiado,23,6135,36.841061,-2.563604,2300,1280.228731,2.078238,22775.98586,...,0.0,0.0,201.238037,190.649105,0.0,65.366956,0.0,0.0,0.000000,457.254098
6051,Kenya,Kajiado,100,6136,36.824311,-2.496446,10000,1470.620210,1.952978,22033.22939,...,0.0,0.0,184.610506,181.852759,0.0,54.473249,0.0,0.0,0.000000,420.936515
6052,Kenya,Kajiado,100,6137,36.824311,-2.399384,10000,1370.762115,1.774332,22489.47934,...,0.0,0.0,200.076368,190.900542,0.0,57.948374,0.0,0.0,0.000000,448.925284


In [466]:
#determining solar potential of month with max demand

df['etc_max']=df.filter(like='IRn_').idxmax(axis=1)

d = df['etc_max']

x = d.str[4:]

x
n = x.astype(int)
 
start_pv = df.columns.get_loc('pvout_1')
pos = start_pv + (n)
pos

colname = df.columns[pos]


for i in range(len(df)):
    df['PV'] = df.loc[:][colname[i]]

df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,month_demand_6,month_demand_7,month_demand_8,month_demand_9,month_demand_10,month_demand_11,month_demand_12,Annual_elec_demand,etc_max,PV
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,784.727948,0.000000,0.0,12.064718,0.0,0.0,0.000000,1630.550146,IRn_1,111.015729
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,836.134605,53.760176,0.0,72.402149,0.0,0.0,0.000000,1836.522279,IRn_1,111.183339
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,900.426239,109.478088,0.0,108.594018,0.0,0.0,146.875422,2214.493818,IRn_1,111.213198
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,912.684748,103.407326,0.0,132.427845,0.0,0.0,249.380666,2386.373117,IRn_1,110.829992
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,585.533899,0.000000,0.0,0.000000,0.0,0.0,0.000000,1278.966248,IRn_1,113.597169


In [467]:
#sizing the PV

In [468]:
#PV size = 1.2 x Max Energy Demand (kWh)/pv potential (kWh/kWp)
#Result scaled by 20\% for safety as the monthly PVOUT value is an average

df['pv_kw'] = (1.2*df['PD_E']*8*30)/df['PV']


df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,month_demand_7,month_demand_8,month_demand_9,month_demand_10,month_demand_11,month_demand_12,Annual_elec_demand,etc_max,PV,pv_kw
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,0.000000,0.0,12.064718,0.0,0.0,0.000000,1630.550146,IRn_1,111.015729,9.012317
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,53.760176,0.0,72.402149,0.0,0.0,0.000000,1836.522279,IRn_1,111.183339,9.435500
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,109.478088,0.0,108.594018,0.0,0.0,146.875422,2214.493818,IRn_1,111.213198,10.241087
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,103.407326,0.0,132.427845,0.0,0.0,249.380666,2386.373117,IRn_1,110.829992,10.702582
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,0.000000,0.0,0.000000,0.0,0.0,0.000000,1278.966248,IRn_1,113.597169,7.325172


In [469]:
#Total energy produced by system per month
#Energy (kWh) = PV Peak Power (kWp) * solar potential (kWh/kWp) 
for i in range (1,13):
    df['E_supply_{}'.format(i)] = df['pv_kw']*df['pvout_{}'.format(i)]

    
#Total energy produced by the system annually
#Total energy supplied given by the sum of the monthly supplies
df['Annual_Energy']=df.filter(like='E_supply_').sum(axis=1)
# df['Max_Energy']=df.filter(like='E_supply_').max(axis=1)


#utilisation factor
df['utilisation_1'] = 100 * df['Annual_elec_demand']/df['Annual_Energy']


df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,E_supply_5,E_supply_6,E_supply_7,E_supply_8,E_supply_9,E_supply_10,E_supply_11,E_supply_12,Annual_Energy,utilisation_1
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,1076.685516,880.957837,854.378068,1000.508975,1140.093277,1065.192974,1061.310341,1313.556326,13692.331437,11.908492
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,1128.437212,921.390986,894.531001,1049.070419,1196.592547,1110.096436,1113.616116,1375.530734,14342.260410,12.804971
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,1220.214477,996.585113,966.604466,1138.944063,1301.359867,1205.682182,1222.168581,1502.817553,15601.592820,14.194024
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,1272.362366,1037.208049,1003.758311,1186.167039,1361.030985,1256.015172,1284.407758,1586.029455,16338.184980,14.606109
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,892.753282,735.314398,718.184772,832.118818,940.649344,890.160948,882.453120,1070.912878,11248.420814,11.370185


In [470]:
#Excess energy per month i.e. the energy not used for irrigation

for i in range (1,13):
    E_Excess = 'E_Excess_{}'.format(i)
    E_Supply = 'E_supply_{}'.format(i)
    demand = 'month_demand_{}'.format(i)
    df[E_Excess] = df[E_Supply] - df[demand]

#electrification demand
#tier 3 demand per month = 1000kWh/12
#assuming 4 people per household
#demand = area being connected * density of people without electricity * demand per/household/month

#connecting 1km^2 around the SPIS
df['tier3'] = df['no_access']*1*1000/(4*12)

    
df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,E_Excess_4,E_Excess_5,E_Excess_6,E_Excess_7,E_Excess_8,E_Excess_9,E_Excess_10,E_Excess_11,E_Excess_12,tier3
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,1119.109848,1076.685516,96.229889,854.378068,1000.508975,1128.028559,1065.192974,1061.310341,1313.556326,269.185883
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,1165.841609,1128.437212,85.256381,840.770826,1049.070419,1124.190397,1110.096436,1113.616116,1375.530734,209.131858
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,1260.675068,1220.214477,96.158875,857.126378,1138.944063,1192.765849,1205.682182,1222.168581,1355.942132,512.402896
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,1317.424317,1272.362366,124.523301,900.350986,1186.167039,1228.603140,1256.015172,1284.407758,1336.648789,491.740801
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,921.058071,892.753282,149.780499,718.184772,832.118818,940.649344,890.160948,882.453120,1070.912878,62.557369


In [471]:
#determining how much of the SPIS excess energy can be consumed by the households/per month

for i in range (1,13):
    E_Excess = 'E_Excess_{}'.format(i)
    amount_supplied = 'amount_supplied_{}'.format(i)
    Tier3 = 'tier3'
    df.loc[df[E_Excess] > df[Tier3], amount_supplied] = df[Tier3] #if supply greater than demand
    df.loc[df[E_Excess] < df[Tier3], amount_supplied] = df[E_Excess] #if demand greater than supply
    
df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,amount_supplied_3,amount_supplied_4,amount_supplied_5,amount_supplied_6,amount_supplied_7,amount_supplied_8,amount_supplied_9,amount_supplied_10,amount_supplied_11,amount_supplied_12
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,269.185883,269.185883,269.185883,96.229889,269.185883,269.185883,269.185883,269.185883,269.185883,269.185883
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,209.131858,209.131858,209.131858,85.256381,209.131858,209.131858,209.131858,209.131858,209.131858,209.131858
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,512.402896,512.402896,512.402896,96.158875,512.402896,512.402896,512.402896,512.402896,512.402896,512.402896
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,491.740801,491.740801,491.740801,124.523301,491.740801,491.740801,491.740801,491.740801,491.740801,491.740801
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,62.557369,62.557369,62.557369,62.557369,62.557369,62.557369,62.557369,62.557369,62.557369,62.557369


In [472]:
#total energy supplied to electrified homes annually
df['total_supplied']=df.filter(like='amount_supplied_').sum(axis=1)

#number of households getting tier 3
df['tier3_hh'] = df['total_supplied']/1000

#new utilisation factor
#adding up energy supplied for electrification and energy used for irrigation
df['utilisation_2'] = 100*(df['Annual_elec_demand'] + df['total_supplied'])/df['Annual_Energy']

df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,amount_supplied_6,amount_supplied_7,amount_supplied_8,amount_supplied_9,amount_supplied_10,amount_supplied_11,amount_supplied_12,total_supplied,tier3_hh,utilisation_2
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,96.229889,269.185883,269.185883,269.185883,269.185883,269.185883,269.185883,3057.274599,3.057275,34.236863
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,85.256381,209.131858,209.131858,209.131858,209.131858,209.131858,209.131858,2385.706823,2.385707,29.439077
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,96.158875,512.402896,512.402896,512.402896,512.402896,512.402896,512.402896,5732.590733,5.732591,50.937649
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,124.523301,491.740801,491.740801,491.740801,491.740801,491.740801,491.740801,5533.672110,5.533672,48.475674
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,62.557369,62.557369,62.557369,62.557369,62.557369,62.557369,62.557369,750.688422,0.750688,18.043908


In [473]:
#system cost
#cost of submersible pump + solar panels = $1109/kW #https://www.waterpumps.co.ke/solar-water-pumps-prices-in-kenya.html
#cost of surface pump + solar panels = $1295/kW #https://www.waterpumps.co.ke/solar-water-pumps-prices-in-kenya.html
#cost drilling a borehole = $833 + $54*depth #https://www.waterlink.co.ke/2021/05/07/borehole-drilling-cost-in-kenya-2022/#:~:text
#cost of water transmission pipeline = $1.14*distance #https://grekkon.com/hdpe-pipes-in-kenya/

#cost excluding the boreholes or the water transmission pipe
df.loc[df['sw'] == 1, 'cost'] = 1295*df['PD_E'] #if using surface water
df.loc[df['sw'] == 0, 'cost'] = 1109*df['PD_E'] #if using ground water

df.head(5)

,ADM0_EN,COUNTY,Area,Main ID,lon,lat,area,elevation,wind_10m,srad_1,...,amount_supplied_7,amount_supplied_8,amount_supplied_9,amount_supplied_10,amount_supplied_11,amount_supplied_12,total_supplied,tier3_hh,utilisation_2,cost
0,Kenya,Wajir,99,1,40.942462,2.016948,9900,246.495982,3.105536,24412.03003,...,269.185883,269.185883,269.185883,269.185883,269.185883,269.185883,3057.274599,3.057275,34.236863,3852.654353
1,Kenya,Wajir,99,2,40.942562,2.114010,9900,266.571416,3.130761,24479.77660,...,209.131858,209.131858,209.131858,209.131858,209.131858,209.131858,2385.706823,2.385707,29.439077,4039.649634
2,Kenya,Mandera,99,3,40.942632,2.211072,9900,286.917205,3.160159,24562.24128,...,512.402896,512.402896,512.402896,512.402896,512.402896,512.402896,5732.590733,5.732591,50.937649,4385.725575
3,Kenya,Mandera,99,4,40.942720,2.308133,9900,320.437701,3.179663,24648.61526,...,491.740801,491.740801,491.740801,491.740801,491.740801,491.740801,5533.672110,5.533672,48.475674,4567.566829
4,Kenya,Wajir,99,5,40.942665,1.628699,9900,175.781640,3.120238,24153.04621,...,62.557369,62.557369,62.557369,62.557369,62.557369,62.557369,750.688422,0.750688,18.043908,3204.235310


In [474]:
#removing protected areas
x = df.loc[df['protected'] == 1]
# x
#removing areas where minimum aquifer productivity is less than peak water daily demand
# y = x.loc[x['aquifer_min'] < x['pcwr_max']]
# y

In [475]:
#filtering useful columns
#county, main id, longitude, latitude
#pcwr, peak demand, pv, energy demand, utilisation 1, electricity supplied, utilisation 2,  cost
# df['pcwr_max']
out = x.filter(items=['COUNTY','Main ID', 'lon','lat','pwcr_per_day', 'PD_E','pv_kw', 'Annual_elec_demand',
                'total_supplied','utilisation_1','cost'])
out.head(5)

,COUNTY,Main ID,lon,lat,pwcr_per_day,PD_E,pv_kw,Annual_elec_demand,total_supplied,utilisation_1,cost
0,Wajir,1,40.942462,2.016948,42256.227871,3.473989,9.012317,1630.550146,3057.274599,11.908492,3852.654353
1,Wajir,2,40.942562,2.114010,44455.112280,3.642606,9.435500,1836.522279,2385.706823,12.804971,4039.649634
2,Mandera,3,40.942632,2.211072,48534.438837,3.954667,10.241087,2214.493818,5732.590733,14.194024,4385.725575
3,Mandera,4,40.942720,2.308133,50682.558592,4.118636,10.702582,2386.373117,5533.672110,14.606109,4567.566829
4,Wajir,5,40.942665,1.628699,34665.039135,2.889301,7.325172,1278.966248,750.688422,11.370185,3204.235310
...,...,...,...,...,...,...,...,...,...,...,...
6047,Laikipia,6132,36.824311,0.221296,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
6048,Laikipia,6133,36.824311,0.318358,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
6049,Laikipia,6134,36.824311,0.415420,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN
6050,Kajiado,6135,36.841061,-2.563604,38589.885345,0.838492,2.083069,457.254098,2564.781278,13.916445,929.887430


In [476]:
#saving
if s == 1:
    out.to_csv(r'/Users/s1682399/Downloads/MSc_main_datasets/Python/results/worst_out.csv', index = False)
elif s == 2:
    out.to_csv(r'/Users/s1682399/Downloads/MSc_main_datasets/Python/results/future_out.csv', index = False)
else:
    out.to_csv(r'/Users/s1682399/Downloads/MSc_main_datasets/Python/results/average_out.csv', index = False)
